<a href="https://colab.research.google.com/github/ha-seungwon/Dacon/blob/main/dna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import random
import os
import numpy as np
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
#from catboost import CatBoostClassifier, Pool

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
#%% [code]
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')


#%% [code]
train=train.drop(['father','mother','gender'],axis=1)
test=test.drop(['father','mother','gender'],axis=1)


def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x


#%% [code]
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)






In [21]:
info=pd.read_csv('snp_info.csv')

In [22]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]


In [23]:
'''
for i in range(len(snp_col)):
  snp_col.append(snp_col[i]+'left')
  snp_col.append(snp_col[i]+'right')'''

"\nfor i in range(len(snp_col)):\n  snp_col.append(snp_col[i]+'left')\n  snp_col.append(snp_col[i]+'right')"

In [24]:
train

,id,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,2,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,2,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,1,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


In [26]:
def dna_percnt(df):
  a=[]
  c=[]
  g=[]
  t=[]
  for i in df.index:
      string=df.iloc[i]['SNP_01':'SNP_15']
      string=''.join(string)
      string=string.replace(" ", "")

      a.append(string.count('A'))
      c.append(string.count('C'))
      g.append(string.count('G'))
  df['A']=a
  df['G']=g
  df['C']=c
  df['A_percent']=df['A']/30
  df['G_percent']=df['G']/30
  df['C_percent']=df['C']/30
  df=df.drop(['A','G','C'],axis=1)
  return df
train_x=dna_percnt(train_x)
test_x=dna_percnt(test_x)
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

#%% [code]
snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

#코드 셀 <undefined>
#%% [code]
train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)



for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])


snp_col.append('trait')

train_x=pd.get_dummies(train_x,columns=['trait'])
test_x=pd.get_dummies(test_x,columns=['trait'])




In [27]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import CategoricalNB           # 01. 나이브 베이즈 분류기-범주형
from sklearn.linear_model import LogisticRegression     # 02. 선형 회귀 모델
from sklearn.svm import SVC                             # 03. 서포트 벡터 머신-범주형
from sklearn.ensemble import RandomForestClassifier     # 04. 랜덤 포레스트 분류기
from sklearn.tree import DecisionTreeClassifier         # 05. 의사결정트리
from sklearn.neighbors import KNeighborsClassifier      # 06. 최근접 이웃 분류기
from sklearn.ensemble import GradientBoostingClassifier # 07. 부스팅-경사하강법
from sklearn.ensemble import AdaBoostClassifier         # 08. 부스팅
from sklearn.ensemble import ExtraTreesClassifier       # 09. 랜덤포레스트+무작위성
from lightgbm import LGBMClassifier                     # 10. lightgbm
from xgboost import XGBClassifier                       # 11. xgboost

# 객체 생성

'''# Training classifiers
clf1 = DecisionTreeClassifier()
clf2 = KNeighborsClassifier()
clf3 = SVC(kernel='rbf', probability=True)
clf4 = CatBoostClassifier()
clf5 = XGBClassifier()
clf6 = LGBMClassifier()
'''
from sklearn.model_selection import LeaveOneOut,cross_val_score
loocv = LeaveOneOut()
clf01 = CategoricalNB()
clf02 = LogisticRegression()
clf03 = SVC(kernel='rbf', probability=True)
clf04 = RandomForestClassifier()
clf05 = DecisionTreeClassifier()
clf06 = KNeighborsClassifier()
clf07 = GradientBoostingClassifier()
clf08 = AdaBoostClassifier()
clf09 = ExtraTreesClassifier()
clf10 = LGBMClassifier()
clf11 = XGBClassifier()
clf12 = CatBoostClassifier()

vo_clf = VotingClassifier(estimators=[('cnb', clf01), ('lr', clf02), ('svc', clf03),('rf',clf04),('dt',clf05),('knn',clf06),
                                      ('gb', clf07), ('ada', clf08), ('et', clf09), ('lgb', clf10), ('xgb', clf11)],
                        voting='soft')

In [ ]:
from sklearn.model_selection import LeaveOneOut,cross_val_score
loocv = LeaveOneOut()
scores = cross_val_score(vo_clf,train_x, train_y,cv = loocv)



print("교차 검증 분할 횟수 : ", len(scores))
print("평균 정확도 : {:.2f}".format(scores.mean()))



In [1]:
scores

NameError: name 'scores' is not defined

In [439]:
#%% [code]
submit = pd.read_csv('./sample_submission.csv')


#%% [code]
submit['class'] = class_le.inverse_transform(xg_pred)


#%% [code]
submit.to_csv('./submit.csv', index=False)




In [440]:
submit['class'].value_counts()

#'''B    85
#A    50
#C    40
#mlp

#B    87
#A    51
#C    37
#xgb

#B    85
#A    51
#C    39
#voting_pred

B    87
A    51
C    37
Name: class, dtype: int64